In [ ]:
!unzip /content/drive/MyDrive/dataset/01_printed_syllable_images.zip -d /content/drive/MyDrive/dataset

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: /content/drive/MyDrive/dataset/syllable/00508191.png  
  inflating: /content/drive/MyDrive/dataset/syllable/00508198.png  
  inflating: /content/drive/MyDrive/dataset/syllable/00508199.png  
  inflating: /content/drive/MyDrive/dataset/syllable/00508204.png  
  inflating: /content/drive/MyDrive/dataset/syllable/00508209.png  
  inflating: /content/drive/MyDrive/dataset/syllable/00508210.png  
  inflating: /content/drive/MyDrive/dataset/syllable/00508214.png  
 extracting: /content/drive/MyDrive/dataset/syllable/00508215.png  
  inflating: /content/drive/MyDrive/dataset/syllable/00508219.png  
  inflating: /content/drive/MyDrive/dataset/syllable/00508220.png  
  inflating: /content/drive/MyDrive/dataset/syllable/00508229.png  
  inflating: /content/drive/MyDrive/dataset/syllable/00508234.png  
  inflating: /content/drive/MyDrive/dataset/syllable/00508235.png  
  inflating: /content/drive/MyDrive/dataset/syllable/00508237.png  
  inflating:

In [ ]:
import numpy as np
import json
import matplotlib.pylab as plt
import sys
from PIL import Image
from keras.utils import to_categorical
from keras import models, layers

#Image Data Open
#with open('C:/Users/원기/Korean/handwriting_data_info1.json','r',encoding='utf-8') as f:
    #Korean_data_hand = json.load(f)

with open('/content/drive/MyDrive/dataset/printed_data_info.json','r',encoding='utf-8') as f:
    Korean_data_print = json.load(f)

with open('/content/drive/MyDrive/dataset/Korean_2350.txt','r',encoding='utf-8') as f:
    Korean_2350 = f.read()
    
Korean_2350 = Korean_2350.split()
print(Korean_2350)
#Training을 위한 Data Save
Korean_id = []
Korean_text = []

#for i,Data in enumerate(Korean_data_hand['annotations']):
 #   if Data['attributes']['type']== '글자(음절)':
  #      if Data['text'] in Korean_2350:
   #         Korean_id.extend([Data['id']])
    #        Korean_text.extend([Data['text']])
            
Korean_id = Korean_id[0:28345]+Korean_id[29602:-631]
Korean_text = Korean_text[0:28345]+Korean_text[29602:-631]

print(Korean_id)
print(Korean_text)

for i,Data in enumerate(Korean_data_print['annotations']):
    if Data['attributes']['type']== '글자(음절)':
        if Data['text'] in Korean_2350:
            Korean_id.extend([Data['id']])
            Korean_text.extend([Data['text']])

#메모리 확보를 위해 file Data 정리
del(Korean_data_print)
del(Korean_2350)

#Train & test data Split
Korean_train_id = Korean_id[0:240000]
Korean_train_text = Korean_text[0:240000]

Korean_test_id = Korean_id[240000:264385]
Korean_test_text = Korean_text[240000:264385]

syllable = list(set(Korean_text))
print('"음절 총 개수: ', len(syllable))

#Training에 필요한 만큼 Data가 있는지 확인
sample=0
for Data in Korean_text:
    if Data == '최':
        sample+=1
print('"최"라는 단어를 사용한 횟수: ', sample)

#메모리 확보를 위해 file Data 정리
del(Korean_id)
del(Korean_text)

#Image File 불러오기
x_train = np.zeros((240000,32,32,3),'uint8')
x_test = np.zeros((24385,32,32,3),'uint8')

for i, ID in enumerate(Korean_train_id):
    if i < 28345:
        Image_addr = 'C:/Users/원기/Korean/01_handwriting_syllable_images/1_syllable/' \
            +str(ID)+'.png'
    elif i < 152432:
        Image_addr = 'C:/Users/원기/Korean/2_syllable/' \
            +str(ID)+'.png'
    else:
        Image_addr = '/content/drive/MyDrive/dataset/syllable' \
            +str(ID)+'.png'        
    Korean_Image = Image.open(Image_addr)
    Korean_Image = Korean_Image.resize((32,32))
    Korean_Image_Array = np.array(Korean_Image,'uint8')
    x_train[i] = Korean_Image_Array

for i,ID in enumerate(Korean_test_id):
    Image_addr = '/content/drive/MyDrive/dataset/syllable' \
        +str(ID)+'.png'
    Korean_Image = Image.open(Image_addr)
    Korean_Image = Korean_Image.resize((32,32))
    Korean_Image_Array = np.array(Korean_Image,'uint8')
    x_test[i] = Korean_Image_Array

#CNN을 위해 array shape 변경 후 0 ~ 1 사이로 값 Normalize
x_train = x_train.astype('float32')/255.0
x_test = x_test.astype('float32')/255.0

print(Korean_train_id)
print(Korean_test_id)

#메모리 확보를 위해 file Data 정리
del(Korean_train_id)
del(Korean_test_id)

#각 음절 Indexing
syllable_to_index = {syllable: index for index, syllable in enumerate(syllable)}
index_to_syllable = {index: syllable for index, syllable in enumerate(syllable)}

label_train = []
label_test = []

for syllables in Korean_train_text:
    if syllable_to_index.get(syllables) is not None:
        label_train.extend([syllable_to_index[syllables]])

for syllables in Korean_test_text:
    if syllable_to_index.get(syllables) is not None:
        label_test.extend([syllable_to_index[syllables]])

print(Korean_train_text)
print(Korean_test_text)
print(label_train)
print(label_test)
#메모리 확보를 위해 file Data 정리


#One - hot encoding
label_train = to_categorical(label_train)
label_test = to_categorical(label_test)

#CNN layer 추가 후 Parameter 설정
CNN = models.Sequential()
CNN.add(layers.Conv2D(128, (3,3), activation='relu',input_shape=(32,32,3)))
CNN.add(layers.MaxPooling2D((2,2)))

CNN.add(layers.Conv2D(256, (3,3), activation='relu'))
CNN.add(layers.MaxPooling2D((2,2)))

CNN.add(layers.Conv2D(512, (3,3), activation='relu'))
CNN.add(layers.Flatten())

CNN.add(layers.Dense(512, activation='relu'))

CNN.add(layers.Dense(2349, activation='softmax'))

#CNN 함수 설정 후 Training
from keras import optimizers
CNN.compile(optimizer = optimizers.RMSprop(lr=0.001),
loss = 'categorical_crossentropy',
metrics = ['accuracy'])

hist = CNN.fit(x_train, label_train,
epochs = 20, batch_size = 128,
validation_split=0.2)

#test accuracy 출력
test_loss, test_acc = CNN.evaluate(x_test, label_test)
print('test_loss:      ',test_loss)
print('test_accuracy:  ',test_acc)

#Epoch당 loss / accuracy plot
loss = hist.history['loss']
acc = hist.history['accuracy']
val_loss = hist.history['val_loss']
val_acc = hist.history['val_accuracy']
epochs = range(1, len(loss)+1)
plt.figure(figsize=(10,7))
plt.subplots_adjust(wspace=0.5)
plt.subplot(1,2,1)
plt.plot(epochs, loss, 'bo-', label='Training loss')
plt.plot(epochs, val_loss, 'rx-', label='Validation loss')
plt.title('Loss')
plt.xlabel('Epochs')
plt.ylabel('loss')
plt.grid()
plt.legend()
plt.subplot(1,2,2)
plt.plot(epochs, acc, 'bo-', label='Training accuracy')
plt.plot(epochs, val_acc, 'rx-', label='Validation accuracy')
plt.title('Accuracy')
plt.xlabel('Epochs')
plt.ylabel('accuracy')
plt.grid()
plt.legend()


['가각간갇갈갉갊감갑값갓갔강갖갗같갚갛개객갠갤갬갭갯갰갱갸갹갼걀걋걍걔걘걜거걱건걷걸걺검겁것겄겅겆겉겊겋게겐겔겜겝겟겠겡겨격겪견겯결겸겹겻겼경곁계곈곌곕곗고곡곤곧골곪곬곯곰곱곳공곶과곽관괄괆괌괍괏광괘괜괠괩괬괭괴괵괸괼굄굅굇굉교굔굘굡굣구국군굳굴굵굶굻굼굽굿궁궂궈궉권궐궜궝궤궷귀귁귄귈귐귑귓규균귤그극근귿글긁금급긋긍긔기긱긴긷길긺김깁깃깅깆깊까깍깎깐깔깖깜깝깟깠깡깥깨깩깬깰깸깹깻깼깽꺄꺅꺌꺼꺽꺾껀껄껌껍껏껐껑께껙껜껨껫껭껴껸껼꼇꼈꼍꼐꼬꼭꼰꼲꼴꼼꼽꼿꽁꽂꽃꽈꽉꽐꽜꽝꽤꽥꽹꾀꾄꾈꾐꾑꾕꾜꾸꾹꾼꿀꿇꿈꿉꿋꿍꿎꿔꿜꿨꿩꿰꿱꿴꿸뀀뀁뀄뀌뀐뀔뀜뀝뀨끄끅끈끊끌끎끓끔끕끗끙끝끼끽낀낄낌낍낏낑나낙낚난낟날낡낢남납낫났낭낮낯낱낳내낵낸낼냄냅냇냈냉냐냑냔냘냠냥너넉넋넌널넒넓넘넙넛넜넝넣네넥넨넬넴넵넷넸넹녀녁년녈념녑녔녕녘녜녠노녹논놀놂놈놉놋농높놓놔놘놜놨뇌뇐뇔뇜뇝뇟뇨뇩뇬뇰뇹뇻뇽누눅눈눋눌눔눕눗눙눠눴눼뉘뉜뉠뉨뉩뉴뉵뉼늄늅늉느늑는늘늙늚늠늡늣능늦늪늬늰늴니닉닌닐닒님닙닛닝닢다닥닦단닫달닭닮닯닳담답닷닸당닺닻닿대댁댄댈댐댑댓댔댕댜더덕덖던덛덜덞덟덤덥덧덩덫덮데덱덴델뎀뎁뎃뎄뎅뎌뎐뎔뎠뎡뎨뎬도독돈돋돌돎돐돔돕돗동돛돝돠돤돨돼됐되된될됨됩됫됬됴두둑둔둘둠둡둣둥둬뒀뒈뒝뒤뒨뒬뒵뒷뒹듀듄듈듐듕드득든듣들듦듬듭듯등듸디딕딘딛딜딤딥딧딨딩딪따딱딴딸땀땁땃땄땅땋때땍땐땔땜땝땟땠땡떠떡떤떨떪떫떰떱떳떴떵떻떼떽뗀뗄뗌뗍뗏뗐뗑뗘뗬또똑똔똘똥똬똴뙈뙤뙨뚜뚝뚠뚤뚫뚬뚱뛔뛰뛴뛸뜀뜁뜅뜨뜩뜬뜯뜰뜸뜹뜻띄띈띌띔띕띠띤띨띰띱띳띵라락란랄람랍랏랐랑랒랖랗래랙랜랠램랩랫랬랭랴략랸럇량러럭런럴럼럽럿렀렁렇레렉렌렐렘렙렛렝려력련렬렴렵렷렸령례롄롑롓로록론롤롬롭롯롱롸롼뢍뢨뢰뢴뢸룀룁룃룅료룐룔룝룟룡루룩룬룰룸룹룻룽뤄뤘뤠뤼뤽륀륄륌륏륑류륙륜률륨륩륫륭르륵른를름릅릇릉릊릍릎리릭린릴림립릿링마막만많맏말맑맒맘맙맛망맞맡맣매맥맨맬맴맵맷맸맹맺먀먁먈먕머먹먼멀멂멈멉멋멍멎멓메멕멘멜멤멥멧멨멩며멱면멸몃몄명몇몌모목몫몬몰몲몸몹못몽뫄뫈뫘뫙뫼묀묄묍묏묑묘묜묠묩묫무묵묶문묻물묽묾뭄뭅뭇뭉뭍뭏뭐뭔뭘뭡뭣뭬뮈뮌뮐뮤뮨뮬뮴뮷므믄믈믐믓미믹민믿밀밂밈밉밋밌밍및밑바박밖밗반받발밝밞밟밤밥밧방밭배백밴밸뱀뱁뱃뱄뱅뱉뱌뱍뱐뱝버벅번벋벌벎범법벗벙벚베벡벤벧벨벰벱벳벴벵벼벽변별볍볏볐병볕볘볜보복볶본볼봄봅봇봉봐봔봤봬뵀뵈뵉뵌뵐뵘뵙뵤뵨부북분붇불붉붊붐붑붓붕붙

ValueError: ignored

error: ignored

(0, 20, 20, 1)


FileNotFoundError: ignored